<a href="https://colab.research.google.com/github/Rakib911Hossan/hate_speech_detection_demo/blob/main/TaskC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# [Hate Speech Identification Shared Task](https://multihate.github.io/): Subtask 1C at [BLP Workshop](https://blp-workshop.github.io/) @IJCNLP-AACL 2025

# This shared task is designed to identify the type of hate, its severity, and the targeted group from social media content.
# In this subtask, given a Bangla text collected from YouTube comments, we need to predict:
# 1. hate_type: Abusive, Sexism, Religious Hate, Political Hate, Profane, or None
# 2. hate_severity: Little to None, Mild, or Severe
# 3. to_whom: Individuals, Organizations, Communities, or Society

### Downloading dataset from github
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_train.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_dev.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_dev_test.tsv

In [ ]:

!pip install transformers
!pip install datasets
!pip install evaluate
# !pip install --upgrade accelerate

In [ ]:
### Importing required libraries
import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional
import pandas as pd
import datasets
import evaluate
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
import torch

import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version


logger = logging.getLogger(__name__)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)


In [ ]:
### Define file paths
train_file = 'blp25_hatespeech_subtask_1C_train.tsv'
validation_file = 'blp25_hatespeech_subtask_1C_dev.tsv'
test_file = 'blp25_hatespeech_subtask_1C_dev_test.tsv'


In [ ]:
train_df = pd.read_csv(train_file, sep='\t')

In [ ]:

# # Shape of dataset
print("Shape:", train_df.shape)

# # Column names
print("Columns:", train_df.columns)

# Data types & non-null counts
print(train_df.info())

# First few rows
print(train_df.head())

# Last few rows
print(train_df.tail())

In [ ]:

# Numeric summary
print(train_df.describe())

# Categorical summary
print(train_df.describe(include='object'))

# Unique values per column
for col in train_df.columns:
    print(f"{col}: {train_df[col].nunique()} unique values")



print(train_df.isnull().sum())  # Count missing values
print(train_df.isna().mean()*100)  # Percentage of missing values



In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:

from transformers import TrainingArguments, EarlyStoppingCallback

# 🎯 OPTIMIZED CONFIGURATION BASED ON YOUR RESULTS
training_args = TrainingArguments(
    output_dir="./xlm_roberta_recovery/",
    overwrite_output_dir=True,

    # 🔧 ADJUSTED LEARNING SCHEDULE - Your model peaked early then overfitted
    learning_rate=10e-6,                    # Lower LR for more stable convergence
    num_train_epochs=8,                    # More epochs with early stopping
    warmup_ratio=0.15,                     # Longer warmup for stability
    lr_scheduler_type="cosine",            # Smoother LR decay
    # 🛡️ STRONGER REGULARIZATION - Combat the overfitting you observed
    weight_decay=0.01,                     # Increase from your 0.01
    max_grad_norm=0.5,                     # Tighter gradient clipping
    dataloader_drop_last=True,             # More consistent batch sizes

    # ✅ KEEP YOUR SUCCESSFUL BATCH CONFIGURATION
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,

    # 🎯 EARLY STOPPING - Prevent the Epoch 4 overfitting you experienced
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    greater_is_better=True,

    # 📊 COMPREHENSIVE MONITORING
    logging_steps=250,                     # More frequent logging
    eval_steps=None,                       # Eval every epoch
    save_total_limit=3,                    # Keep only best 3 checkpoints

    # 🔧 SYSTEM OPTIMIZATIONS
    report_to=None,
    dataloader_num_workers=2,
    fp16=True,                            # Mixed precision for efficiency
    group_by_length=True,                 # Batch similar lengths together
)

# 🛑 MANDATORY EARLY STOPPING - Based on your overfitting pattern
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2,            # Stop after 2 epochs without improvement
    early_stopping_threshold=0.001        # Minimum improvement threshold
)



# 🎯 OPTIMIZED DATA PARAMETERS
max_train_samples = None
max_eval_samples = None
max_predict_samples = None
max_seq_length = 512
batch_size = 8


In [ ]:
transformers.utils.logging.set_verbosity_info()

log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f" distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer

class MultiTaskClassifier(nn.Module):
    def __init__(self, model_name, num_labels_dict):
        super(MultiTaskClassifier, self).__init__()
        self.encoder = AutoModel.from_pretrained(model_name)
        hidden_size = self.encoder.config.hidden_size  # XLM-R hidden dim = 768

        # Separate heads for each task
        self.hate_type_head = nn.Linear(hidden_size, num_labels_dict['hate_type'])
        self.hate_severity_head = nn.Linear(hidden_size, num_labels_dict['hate_severity'])
        self.to_whom_head = nn.Linear(hidden_size, num_labels_dict['to_whom'])

    def forward(self, input_ids, attention_mask=None):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]  # CLS token representation

        hate_type_logits = self.hate_type_head(pooled_output)
        hate_severity_logits = self.hate_severity_head(pooled_output)
        to_whom_logits = self.to_whom_head(pooled_output)

        return {
            "hate_type": hate_type_logits,
            "hate_severity": hate_severity_logits,
            "to_whom": to_whom_logits
        }

# Setup
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

num_labels_dict = {
    "hate_type": 5,        # 5 classes
    "hate_severity": 3,    # 3 classes
    "to_whom": 4           # 4 classes
}

model = MultiTaskClassifier(model_name, num_labels_dict)


In [ ]:

set_seed(42)

In [ ]:
import pandas as pd
import numpy as np

# First, let's load and examine the basic structure
print("=== LOADING AND BASIC INSPECTION ===")
try:
    train_df = pd.read_csv(train_file, sep='\t')
    print(f"✅ Successfully loaded training data: {train_df.shape}")
except Exception as e:
    print(f"❌ Error loading training data: {e}")
    train_df = None

if train_df is not None:
    print("\n=== COLUMN INFORMATION ===")
    print(f"Total columns: {len(train_df.columns)}")
    print(f"Column names: {list(train_df.columns)}")
    print(f"Column data types:")
    for col in train_df.columns:
        print(f"  {col}: {train_df[col].dtype}")

    print("\n=== FIRST FEW ROWS ===")
    print(train_df.head())

    print("\n=== NUMERIC SUMMARY ===")
    numeric_cols = train_df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(train_df[numeric_cols].describe())
    else:
        print("No numeric columns found")

    print("\n=== CATEGORICAL SUMMARY ===")
    categorical_cols = train_df.select_dtypes(include=['object']).columns
    if len(categorical_cols) > 0:
        print(train_df[categorical_cols].describe())
    else:
        print("No categorical columns found")

    print("\n=== UNIQUE VALUES PER COLUMN ===")
    for col in train_df.columns:
        unique_count = train_df[col].nunique()
        print(f"{col}: {unique_count} unique values")

        # Show unique values for columns with reasonable number of unique values
        if unique_count <= 20:
            unique_vals = train_df[col].unique()
            print(f"  Values: {unique_vals}")
        elif unique_count <= 100:
            # Show first 10 unique values for medium-sized categories
            unique_vals = train_df[col].unique()[:10]
            print(f"  First 10 values: {unique_vals}...")

    print("\n=== MISSING VALUES ANALYSIS ===")
    missing_counts = train_df.isnull().sum()
    missing_percentages = train_df.isna().mean() * 100

    print("Missing value counts:")
    for col in train_df.columns:
        count = missing_counts[col]
        percentage = missing_percentages[col]
        print(f"  {col}: {count} ({percentage:.1f}%)")

    print("\n=== COLUMN NAME MATCHING ANALYSIS ===")
    # Check for potential matches with expected columns
    expected_columns = ['hate_type', 'hate_severity', 'to_whom']
    actual_columns = [col.lower().strip() for col in train_df.columns]

    print("Looking for matches with expected columns:")
    for expected in expected_columns:
        print(f"\nLooking for '{expected}':")

        # Exact match
        if expected in actual_columns:
            original_col = train_df.columns[actual_columns.index(expected)]
            print(f"  ✅ Exact match found: '{original_col}'")
        else:
            # Partial matches
            partial_matches = []
            for i, actual in enumerate(actual_columns):
                if expected.replace('_', '') in actual.replace('_', '') or actual in expected:
                    partial_matches.append(train_df.columns[i])

            if partial_matches:
                print(f"  🔍 Possible matches: {partial_matches}")
            else:
                print(f"  ❌ No matches found")

    print("\n=== SUGGESTED COLUMN MAPPING ===")
    # Create mapping suggestions
    suggestions = {}

    # Common variations for hate speech datasets
    hate_type_patterns = ['type', 'class', 'category', 'hate', 'label']
    severity_patterns = ['severity', 'level', 'intensity', 'degree']
    target_patterns = ['target', 'whom', 'directed', 'group', 'victim']

    for col in train_df.columns:
        col_lower = col.lower().strip()

        # Check hate_type patterns
        for pattern in hate_type_patterns:
            if pattern in col_lower and 'hate_type' not in suggestions:
                suggestions['hate_type'] = col
                break

        # Check severity patterns
        for pattern in severity_patterns:
            if pattern in col_lower and 'hate_severity' not in suggestions:
                suggestions['hate_severity'] = col
                break

        # Check target patterns
        for pattern in target_patterns:
            if pattern in col_lower and 'to_whom' not in suggestions:
                suggestions['to_whom'] = col
                break

    print("Suggested mappings:")
    for expected, suggested in suggestions.items():
        print(f"  {expected} -> '{suggested}'")

    # Show unmapped columns
    mapped_cols = set(suggestions.values())
    unmapped_cols = [col for col in train_df.columns if col not in mapped_cols]
    if unmapped_cols:
        print(f"\nUnmapped columns: {unmapped_cols}")

    print("\n=== SAMPLE VALUES FOR KEY COLUMNS ===")
    # Show sample values for likely label columns
    for col in train_df.columns:
        if train_df[col].nunique() <= 50:  # Likely categorical
            print(f"\nSample values for '{col}':")
            value_counts = train_df[col].value_counts().head(10)
            print(value_counts)

    print("\n=== RECOMMENDED NEXT STEPS ===")
    print("Based on this analysis:")
    print("1. Check the 'Suggested mappings' section above")
    print("2. Update your column names in the mapping code")
    print("3. Example fix:")

    if suggestions:
        print("   # Instead of:")
        print("   # train_df['hate_type'] = train_df['hate_type'].map(hate_type2id)...")
        print("   # Use:")
        for expected, actual in suggestions.items():
            print(f"   train_df['{actual}'] = train_df['{actual}'].map({expected}2id).fillna(-1).astype(int)")
    else:
        print("   # No automatic suggestions found. Please manually check column names.")

else:
    print("Cannot proceed with analysis due to data loading error.")

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Define label mapping dictionaries based on your data
hate_type2id = {
    'Abusive': 0,
    'Political Hate': 1,
    'Profane': 2,
    'Religious Hate': 3,
    'Sexism': 4
}

hate_severity2id = {
    'Little to None': 0,
    'Mild': 1,
    'Severe': 2
}

to_whom2id = {
    'Individual': 0,
    'Organization': 1,
    'Community': 2,
    'Society': 3
}

# Create reverse mappings for reference
id2hate_type = {v: k for k, v in hate_type2id.items()}
id2hate_severity = {v: k for k, v in hate_severity2id.items()}
id2to_whom = {v: k for k, v in to_whom2id.items()}

print("Label mappings created:")
print(f"Hate types: {hate_type2id}")
print(f"Hate severity: {hate_severity2id}")
print(f"Target groups: {to_whom2id}")

# Load datasets
print("\nLoading datasets...")
train_df = pd.read_csv(train_file, sep='\t')
valid_df = pd.read_csv(validation_file, sep='\t')
test_df = pd.read_csv(test_file, sep='\t')

print(f"Train shape: {train_df.shape}")
print(f"Validation shape: {valid_df.shape}")
print(f"Test shape: {test_df.shape}")

# Function to safely apply label mapping
def apply_label_mapping(df, column_name, mapping_dict, dataset_name):
    """Apply label mapping with detailed logging"""
    if column_name not in df.columns:
        print(f"\n⚠️  Column '{column_name}' not found in {dataset_name} dataset")
        print(f"Available columns: {list(df.columns)}")
        # Add dummy column with -1 values for missing labels (common in test sets)
        df[column_name] = -1
        print(f"Added dummy column '{column_name}' with -1 values")
        return df

    print(f"\nProcessing {column_name} in {dataset_name}...")

    # Check original distribution
    original_counts = df[column_name].value_counts(dropna=False)
    print(f"Original {column_name} distribution:")
    print(original_counts)

    # Apply mapping
    df[column_name] = df[column_name].map(mapping_dict).fillna(-1).astype(int)

    # Check final distribution
    final_counts = df[column_name].value_counts(dropna=False)
    print(f"Mapped {column_name} distribution:")
    print(final_counts)

    return df

# Apply mappings to all datasets
datasets = [
    (train_df, "train"),
    (valid_df, "validation"),
    (test_df, "test")
]

# Check test dataset structure first
print(f"\nTest dataset columns: {list(test_df.columns)}")
print("Note: Test datasets often don't include labels for prediction tasks")

for df, name in datasets:
    print(f"\n{'='*20} Processing {name} dataset {'='*20}")

    # Apply each mapping (will handle missing columns gracefully)
    apply_label_mapping(df, 'hate_type', hate_type2id, name)
    apply_label_mapping(df, 'hate_severity', hate_severity2id, name)
    apply_label_mapping(df, 'to_whom', to_whom2id, name)

# Convert to HuggingFace datasets
print("\n" + "="*50)
print("Converting to HuggingFace datasets...")

train_ds = Dataset.from_pandas(train_df)
valid_ds = Dataset.from_pandas(valid_df)
test_ds = Dataset.from_pandas(test_df)

# Create DatasetDict
raw_datasets = DatasetDict({
    "train": train_ds,
    "validation": valid_ds,
    "test": test_ds
})

print("✅ Dataset creation successful!")
print(f"Train samples: {len(train_ds)}")
print(f"Validation samples: {len(valid_ds)}")
print(f"Test samples: {len(test_ds)}")



In [ ]:
# Check test dataset size
print("=== TEST DATASET INFO ===")
print(f"Number of test samples: {len(test_df['id'])}")
print(f"Test dataset shape: {test_df.shape}")
print(f"Test dataset columns: {list(test_df.columns)}")

print("\n=== EXTRACTING UNIQUE LABELS ===")

# Since you have multiple label columns (multi-task), we need to handle each separately
label_columns = ['hate_type', 'hate_severity', 'to_whom']

# Dictionary to store label information for each task
label_info = {}

for label_col in label_columns:
    print(f"\n--- {label_col.upper()} LABELS ---")

    # Extract unique labels from training set (excluding -1 which represents missing values)
    unique_labels = raw_datasets["train"].unique(label_col)
    print(f"All unique values (including -1 for missing): {unique_labels}")

    # Filter out -1 (missing values) to get actual labels
    actual_labels = [label for label in unique_labels if label != -1]
    actual_labels.sort()  # Sort for consistency

    print(f"Actual labels (excluding missing): {actual_labels}")
    print(f"Number of unique labels: {len(actual_labels)}")

    # Store information
    label_info[label_col] = {
        'labels': actual_labels,
        'num_labels': len(actual_labels),
        'all_values': unique_labels
    }

    # Show label distribution in training set
    train_df_temp = raw_datasets["train"].to_pandas()
    label_dist = train_df_temp[label_col].value_counts().sort_index()
    print(f"Label distribution in training set:")
    for value, count in label_dist.items():
        if value == -1:
            print(f"  Missing (-1): {count}")
        else:
            # Get original label name
            if label_col == 'hate_type':
                original_name = id2hate_type.get(value, f"Unknown({value})")
            elif label_col == 'hate_severity':
                original_name = id2hate_severity.get(value, f"Unknown({value})")
            else:  # to_whom
                original_name = id2to_whom.get(value, f"Unknown({value})")
            print(f"  {original_name} ({value}): {count}")

print("\n=== SUMMARY ===")
print("Label information for each task:")
for task, info in label_info.items():
    print(f"{task}:")
    print(f"  Number of classes: {info['num_labels']}")
    print(f"  Label range: {min(info['labels'])} to {max(info['labels'])}")

# Total number of labels across all tasks (if you need it)
total_labels = sum(info['num_labels'] for info in label_info.values())
print(f"\nTotal labels across all tasks: {total_labels}")

# Create variables for easy access (commonly used in model setup)
num_hate_types = label_info['hate_type']['num_labels']
num_hate_severities = label_info['hate_severity']['num_labels']
num_to_whom = label_info['to_whom']['num_labels']

print(f"\nVariables for model configuration:")
print(f"num_hate_types = {num_hate_types}")
print(f"num_hate_severities = {num_hate_severities}")
print(f"num_to_whom = {num_to_whom}")

# If you need a single 'num_labels' variable (common for single-task models)
# You would typically use this for the main classification task
num_labels = num_hate_types  # Assuming hate_type is your primary task
print(f"num_labels (primary task - hate_type) = {num_labels}")

print("\n=== VERIFICATION ===")
# Verify our mappings are consistent
print("Checking consistency between mappings and extracted labels:")
for task, info in label_info.items():
    if task == 'hate_type':
        expected_labels = list(range(len(hate_type2id)))
    elif task == 'hate_severity':
        expected_labels = list(range(len(hate_severity2id)))
    else:  # to_whom
        expected_labels = list(range(len(to_whom2id)))

    if set(info['labels']) == set(expected_labels):
        print(f"✅ {task}: Mapping consistent")
    else:
        print(f"❌ {task}: Mapping inconsistent!")
        print(f"   Expected: {expected_labels}")
        print(f"   Found: {info['labels']}")

print("\n=== DATASET SPLITS INFO ===")
for split_name, dataset in raw_datasets.items():
    print(f"{split_name.capitalize()} set: {len(dataset)} samples")

    # Show missing value percentages for each label column
    df_temp = dataset.to_pandas()
    for col in label_columns:
        if col in df_temp.columns:
            missing_count = (df_temp[col] == -1).sum()
            missing_pct = (missing_count / len(df_temp)) * 100
            print(f"  {col} missing: {missing_count} ({missing_pct:.1f}%)")

In [ ]:
# Define model name
model_name = "bert-base-multilingual-cased"  # or your preferred model

# For multi-task, we need custom config with multiple label counts
config = AutoConfig.from_pretrained(
    model_name,
    num_labels=num_hate_types,  # Primary task (hate_type)
    finetuning_task=None,
    cache_dir=None,
    revision="main",
    use_auth_token=None,
)

# Add custom attributes for multi-task
config.num_hate_types = num_hate_types
config.num_hate_severities = num_hate_severities
config.num_to_whom = num_to_whom

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=None,
    use_fast=True,
    revision="main",
    use_auth_token=None,
)

# For multi-task, you'll need a custom model instead of AutoModelForSequenceClassification
# But if you want to keep it simple, use the primary task (hate_type) for now:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    from_tf=bool(".ckpt" in model_name),
    config=config,
    cache_dir=None,
    revision="main",
    use_auth_token=None,
    ignore_mismatched_sizes=False,
)

In [ ]:
# Preprocessing the raw_datasets
non_label_column_names = [name for name in raw_datasets["train"].column_names if name not in ["hate_type", "hate_severity", "to_whom"]]
sentence1_key = "text"  # Your text column

# Padding strategy
padding = "max_length"

# For multi-task, skip the label mapping part since we handle multiple labels
max_seq_length = min(128, tokenizer.model_max_length)

def preprocess_function(examples):
    # Tokenize the texts
    result = tokenizer(examples[sentence1_key], padding=padding, max_length=max_seq_length, truncation=True)

    # Keep all label columns as they are
    result["hate_type"] = examples["hate_type"]
    result["hate_severity"] = examples["hate_severity"]
    result["to_whom"] = examples["to_whom"]

    return result

raw_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    load_from_cache_file=True,
    desc="Running tokenizer on dataset",
)

# Training data
train_dataset = raw_datasets["train"]
if 'max_train_samples' in locals() and max_train_samples is not None:
    train_dataset = train_dataset.select(range(min(len(train_dataset), max_train_samples)))

# Validation data
eval_dataset = raw_datasets["validation"]
if 'max_eval_samples' in locals() and max_eval_samples is not None:
    eval_dataset = eval_dataset.select(range(min(len(eval_dataset), max_eval_samples)))

# Test data
predict_dataset = raw_datasets["test"]
if 'max_predict_samples' in locals() and max_predict_samples is not None:
    predict_dataset = predict_dataset.select(range(min(len(predict_dataset), max_predict_samples)))

# Multi-task compute metrics
def compute_metrics(p: EvalPrediction):
    # For single task (hate_type only)
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

# Data Collator
data_collator = default_data_collator

# Remove ID column for training
train_dataset = train_dataset.remove_columns("id")
eval_dataset = eval_dataset.remove_columns("id")

# Since using AutoModelForSequenceClassification, we need to rename primary label
train_dataset = train_dataset.rename_column("hate_type", "labels")
eval_dataset = eval_dataset.rename_column("hate_type", "labels")

# Initialize Trainer (same as before)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
# Filter out samples with missing labels (-1) before training
print("Filtering out samples with missing labels...")

# Filter training dataset
print(f"Original train dataset size: {len(train_dataset)}")
train_dataset = train_dataset.filter(lambda x: x['labels'] != -1)
print(f"Filtered train dataset size: {len(train_dataset)}")

# Filter validation dataset
print(f"Original eval dataset size: {len(eval_dataset)}")
eval_dataset = eval_dataset.filter(lambda x: x['labels'] != -1)
print(f"Filtered eval dataset size: {len(eval_dataset)}")

# Now train the model
train_result = trainer.train()
metrics = train_result.metrics
max_train_samples = (
    max_train_samples if 'max_train_samples' in locals() and max_train_samples is not None else len(train_dataset)
)
metrics["train_samples"] = min(max_train_samples, len(train_dataset))

# Saving the model and metrics
trainer.save_model()
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

# Evaluating on validation data
logger.info("*** Evaluate ***")
metrics = trainer.evaluate(eval_dataset=eval_dataset)
max_eval_samples = (
    max_eval_samples if 'max_eval_samples' in locals() and max_eval_samples is not None else len(eval_dataset)
)
metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

# Predicting the test data
logger.info("*** Predict ***")
ids = predict_dataset['id']
predict_dataset = predict_dataset.remove_columns("id")
predictions = trainer.predict(predict_dataset, metric_key_prefix="predict").predictions
predictions = np.argmax(predictions, axis=1)

# Create output file
output_predict_file = os.path.join(training_args.output_dir, f"hate_type_predictions.tsv")
if trainer.is_world_process_zero():
    with open(output_predict_file, "w") as writer:
        logger.info(f"***** Predict results *****")
        writer.write("id\thate_type\tmodel\n")
        for index, item in enumerate(predictions):
            # Convert prediction to original label
            hate_type_label = id2hate_type[item]
            writer.write(f"{ids[index]}\t{hate_type_label}\t{model_name}\n")

print(f"First ID: {ids[0]}")

# Saving the model card
kwargs = {"finetuned_from": model_name, "tasks": "text-classification"}
trainer.create_model_card(**kwargs)

# Create zip file
!zip hate_predictions.zip {output_predict_file}

In [ ]:
### Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Apply preprocessing
tokenized_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

In [ ]:
### Custom compute_metrics function
def compute_metrics(eval_pred):
    predictions = eval_pred.predictions
    label_ids = eval_pred.label_ids

    # Calculate accuracy for each task
    hate_type_preds = np.argmax(predictions[0], axis=1)
    severity_preds = np.argmax(predictions[1], axis=1)
    to_whom_preds = np.argmax(predictions[2], axis=1)

    hate_type_acc = accuracy_score(label_ids[0], hate_type_preds)
    severity_acc = accuracy_score(label_ids[1], severity_preds)
    to_whom_acc = accuracy_score(label_ids[2], to_whom_preds)

    # Macro average accuracy
    avg_acc = (hate_type_acc + severity_acc + to_whom_acc) / 3

    return {
        'hate_type_accuracy': hate_type_acc,
        'severity_accuracy': severity_acc,
        'to_whom_accuracy': to_whom_acc,
        'average_accuracy': avg_acc
    }